# Baseline Annotation of Observation based on trained spaCy (2.0.18) model 


<fig size= "4">
This notebook provides a baseline annotation suggestion by using refined named entity recognition that we provided using training a deep learning method by prodigy.  </p>
</fig size= "4">  

In [1]:
import spacy 
from spacy import displacy 

In [2]:
from __future__ import print_function
from ipywidgets import interact
import ipywidgets as widgets
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro","collapse_phrases":False}

In [4]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('max_colwidth', 260)
pd.set_option('display.max_rows', 500)

## Read the Data

In [5]:
importVersion = '013'

In [6]:

path= '../data/01_df_v{0}.pickle'.format(importVersion)# Put the path of the data in your local machine here, consider the letter "r" before the path
dfAstroNova = pd.read_pickle(path)

In [7]:
# Sort the data based on the chapters of the book 
dfAstroNova['chapter'] = dfAstroNova.chapter.replace("appendix b",np.nan).astype(float)  
dfAstroNova = dfAstroNova.rename_axis('MyIdx').sort_values(by = ['chapter', 'MyIdx'], ascending = [True, True])
dfAstroNova.chapter.fillna('appendix b', inplace=True)

In [8]:
dfAstroNova.reset_index(inplace=True)
dfAstroNova=dfAstroNova.drop("MyIdx",axis=1,inplace=False)
dfAstroNova=dfAstroNova.drop("html",axis=1)

In [9]:
#dfAstroNova['paragraph'] = dfAstroNova.index
dfAstroNova.insert(loc=1, column='paragraph', value=dfAstroNova.index)


In [10]:
type(dfAstroNova)

pandas.core.frame.DataFrame

In [11]:
dfAstroNova.head()

,text,paragraph,links,italic,chapter,graphic,table,marginal,sentences,tagged
0,Chapter 1,0,[],[],1,[],[],[],[Chapter 1],"[[(Chapter, None), (1, NUM)]]"
1,"On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.",1,[],[],1,[],[],[],"[On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.]","[[(On, None), (the, None), (distinction, None), (between, None), (the, None), (first, None), (motion, None), (and, None), (the, None), (second, None), (or, None), (proper, None), (motions, None), (and, None), (in, None), (the, None), (proper, None), (motio..."
2,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...",2,[],[],1,[],[],"[ Terms: 1. The first motion is that of the whole heaven and of all its stars from the east through the lowest part of the heavens to the east, in the period of 24 hours; in the present diagram, ABCD. 2. The second motions are those of the individual plane...","[The testimony of the ages confirms that the motions of the planets are orbicular., Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles., For among figures it is circles, and among bodies the heavens...","[[(The, None), (testimony, None), (of, None), (the, None), (ages, None), (confirms, None), (that, None), (the, None), (motions, None), (of, None), (the, None), (planets, None), (are, None), (orbicular, None)], [(Reason, None), (having, None), (borrowed, No..."
3,"It is just this from which astronomy arose among humans. Astronomy's aim is considered to be to show why the stars' motions appear to be irregular on earth, despite their being exceedingly well ordered in heaven, and to investigate the specific circles whe...",3,[],[],1,[ ch 1 gr 1],[],[],"[It is just this from which astronomy arose among humans., Astronomy's aim is considered to be to show why the stars' motions appear to be irregular on earth, despite their being exceedingly well ordered in heaven, and to investigate the specific circles w...","[[(It, None), (is, None), (just, None), (this, None), (from, None), (which, None), (astronomy, None), (arose, None), (among, None), (humans, None)], [(Astronomy, None), (s, None), (aim, None), (is, None), (considered, None), (to, None), (be, None), (to, No..."
4,"Before the distinction between the first motion(1) and the second motions(2) was established, people noted (in contemplating the sun, moon and stars) that their diurnal paths were visually very nearly equivalent to circles. These were, however, entwined on...",4,[],[(such],1,[],[],[ 2],"[Before the distinction between the first motion(1) and the second motions(2) was established, people noted (in contemplating the sun, moon and stars) that their diurnal paths were visually very nearly equivalent to circles., These were, however, entwined ...","[[(Before, None), (the, None), (distinction, None), (between, None), (the, None), (first, None), (motion, None), (1, NUM), (and, None), (the, None), (second, None), (motions, None), (2, NUM), (was, None), (established, None), (people, None), (noted, None),..."


In [12]:
dfAstroNova.tail()

,text,paragraph,links,italic,chapter,graphic,table,marginal,sentences,tagged
1600,,1600,[],[],appendix b,[],[ appendix B t 2],[],[],[]
1601,"Kepler's table, for comparison",1601,[],[],appendix b,[],[ appendix B t 3],[],"[Kepler's table, for comparison]","[[(Kepler, None), (s, None), (table, None), (for, None), (comparison, None)]]"
1602,"Mars from Earth to be a little greater than half of the distance of the Sun from Earth. Kepler computed the diurnal latitudinal parallax of Mars to be 3' 28"", from which he determined the latitude of Mars without parallax to be 2° 25'. Since Mars's distanc...",1602,"[{'#fn6': ' Kepler's observations yield topocentric horizontal coordinates delineated in azimuthal and altitudinal or zenith distance. The transformation of the measurements to geocentric, ecliptic longitude and altitude, or equatorial right ascension and ...",[],appendix b,[],[],[],"[Mars from Earth to be a little greater than half of the distance of the Sun from Earth., Kepler computed the diurnal latitudinal parallax of Mars to be 3' 28"", from which he determined the latitude of Mars without parallax to be 2° 25'., Since Mars's dist...","[[(Mars, None), (from, None), (Earth, None), (to, None), (be, None), (a, None), (little, None), (greater, None), (than, None), (half, None), (of, None), (the, None), (distance, None), (of, None), (the, None), (Sun, None), (from, None), (Earth, None)], [(Ke..."
1603,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...",1603,[],[],appendix b,[ appendix b gr 1],[],[],"[As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system., The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively., The deviation of elongation between Mars and t...","[[(As, None), (shown, None), (in, None), (Figure, None), (2B, None), (the, None), (Sun, None), (is, None), (assumed, None), (to, None), (be, None), (at, None), (the, None), (center, None), (of, None), (the, None), (planetary, None), (system, None)], [(The,..."
1604,"The Sun moved westward and its longitude decreased from the time of observation to its position opposite to Mars. Therefore, the time of opposition is 17 hours 20 minutes before March 29, at 21:43, the time when the observation was made. Kepler determined ...",1604,[],[4<sup>h</sup> 23<sup>m </sup>],appendix b,[],[],[],"[The Sun moved westward and its longitude decreased from the time of observation to its position opposite to Mars., Therefore, the time of opposition is 17 hours 20 minutes before March 29, at 21:43, the time when the observation was made., Kepler determin...","[[(The, None), (Sun, None), (moved, None), (westward, None), (and, None), (its, None), (longitude, None), (decreased, None), (from, None), (the, None), (time, None), (of, None), (observation, None), (to, None), (its, None), (position, None), (opposite, Non..."


In [13]:
df=dfAstroNova.reset_index().text.str.split('.',expand=True).stack().reset_index(level=-1,drop=True)
type(df)

pandas.core.series.Series

In [14]:
 # Create lists to fill with values
l_col1 = []
l_col2 = []
l_col3 = []
l_col4 = []

# iterrate over each row and fill our lists
for ix, row in dfAstroNova.iterrows():
    for value in row['sentences']:
        l_col1.append(row['text'])
        l_col2.append(row['paragraph'])
        l_col3.append(value)
        l_col4.append(row['chapter'])
       
# Create new dataframe from the two lists
df= pd.DataFrame({'Chapter': l_col4,'ParaIndex': l_col2,'Paragraph': l_col1,'Sentence': l_col3 ,
                          })


In [15]:
len(df)

6699

In [16]:
df.head(20)

,Chapter,ParaIndex,Paragraph,Sentence
0,1,0,Chapter 1,Chapter 1
1,1,1,"On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.","On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality."
2,1,2,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...",The testimony of the ages confirms that the motions of the planets are orbicular.
3,1,2,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...","Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles."
4,1,2,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...","For among figures it is circles, and among bodies the heavens, that are considered the most perfect."
5,1,2,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...","However, when experience seems to teach something different to those who pay careful attention, namely, that the planets deviate from a simple circular path, it gives rise to a powerful sense of wonder, which at length drives people to look into causes."
6,1,3,"It is just this from which astronomy arose among humans. Astronomy's aim is considered to be to show why the stars' motions appear to be irregular on earth, despite their being exceedingly well ordered in heaven, and to investigate the specific circles whe...",It is just this from which astronomy arose among humans.
7,1,3,"It is just this from which astronomy arose among humans. Astronomy's aim is considered to be to show why the stars' motions appear to be irregular on earth, despite their being exceedingly well ordered in heaven, and to investigate the specific circles whe...","Astronomy's aim is considered to be to show why the stars' motions appear to be irregular on earth, despite their being exceedingly well ordered in heaven, and to investigate the specific circles whereby the stars may be moved, so that by their aid the pos..."
8,1,4,"Before the distinction between the first motion(1) and the second motions(2) was established, people noted (in contemplating the sun, moon and stars) that their diurnal paths were visually very nearly equivalent to circles. These were, however, entwined on...","Before the distinction between the first motion(1) and the second motions(2) was established, people noted (in contemplating the sun, moon and stars) that their diurnal paths were visually very nearly equivalent to circles."
9,1,4,"Before the distinction between the first motion(1) and the second motions(2) was established, people noted (in contemplating the sun, moon and stars) that their diurnal paths were visually very nearly equivalent to circles. These were, however, entwined on...","These were, however, entwined one upon another like yarn on a ball, and the circles were for the most part smaller(3) circles of the sphere, rarely the greatest(4) (such"


here we also add a index counter for the sentences

In [17]:
df.insert(loc=2, column='SentIndex', value=df.index)

In [18]:
df.head()

,Chapter,ParaIndex,SentIndex,Paragraph,Sentence
0,1,0,0,Chapter 1,Chapter 1
1,1,1,1,"On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.","On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality."
2,1,2,2,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...",The testimony of the ages confirms that the motions of the planets are orbicular.
3,1,2,3,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...","Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles."
4,1,2,4,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...","For among figures it is circles, and among bodies the heavens, that are considered the most perfect."


In [19]:
df.tail()

,Chapter,ParaIndex,SentIndex,Paragraph,Sentence
6694,appendix b,1603,6694,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...","To find the position of Mars at opposition, Kepler computed the angular distance that Mars and Earth—now substituting the place of the Sun—moved during 17 hours 20 minutes; Mars moved eastward about 16' 20"" and the Sun westward about 42' 18""."
6695,appendix b,1603,6695,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...","Accordingly, Kepler determined the longitude of Mars at opposition to be 198° 37' 50"" from which he subtracted about 39"" in order to correct Mars's orbit; he got 198° 37' 10"" (18° 37' 10"" Libra)."
6696,appendix b,1604,6696,"The Sun moved westward and its longitude decreased from the time of observation to its position opposite to Mars. Therefore, the time of opposition is 17 hours 20 minutes before March 29, at 21:43, the time when the observation was made. Kepler determined ...",The Sun moved westward and its longitude decreased from the time of observation to its position opposite to Mars.
6697,appendix b,1604,6697,"The Sun moved westward and its longitude decreased from the time of observation to its position opposite to Mars. Therefore, the time of opposition is 17 hours 20 minutes before March 29, at 21:43, the time when the observation was made. Kepler determined ...","Therefore, the time of opposition is 17 hours 20 minutes before March 29, at 21:43, the time when the observation was made."
6698,appendix b,1604,6698,"The Sun moved westward and its longitude decreased from the time of observation to its position opposite to Mars. Therefore, the time of opposition is 17 hours 20 minutes before March 29, at 21:43, the time when the observation was made. Kepler determined ...","Kepler determined the time of opposition on March 28, 4h 23m AM, old style."


# Upload customized spaCy model provioded by our training data in prodigy

In [20]:
#path= '../data/Model_V17' for V01
path= '../data/Model_U35'
nlp=spacy.load(path)

In [21]:
def entity(x):
    a=[]
    b=[]
    for ent in nlp(x).ents:
        a +=ent.text,
        b +=ent.label_,
    c=list(zip(a,b))
    return c

def attribute(x):
    a=[]
    b=[]
    c=[]
    d=[]
    e=[]
    for token in nlp(x):
        a +=token.text,
        b += token.pos_,
        c += token.tag_,
        d += token.dep_,
        e += token.lemma_
    z=list(zip(a,b,c,d,e))
    return z

def satz_analytic2(satz):
#    merkmal = dict({"satz":satz.text})
    merkmal=dict({})
# search for main verb
    for t in satz:
        if t.dep_ == "ROOT":
            r=t.head.text
            merkmal.update({"act":r})
# subject and object related to verb
    for t in satz:
        if t.dep_ == "dobj" and merkmal["act"]==t.head.text:
             merkmal.update({"obj":''.join(w.text_with_ws for w in t.subtree)})
        if t.dep_ == "nsubj" and merkmal["act"]==t.head.text:
            merkmal.update({"subject":t.text.lower()})

    return(merkmal)

def NerFeature(x):
    list_en=[]
    N=3
    X=list(0 for i in range(0,N)) 
    if x is not None:
        for idx in range(0,len(x)):
            list_en.append(x[idx][1])
            result_0=any([lst in list_en for lst in ["DATE"]])
            if result_0:
                X[0]=1
            result_1=any([lst in list_en for lst in ["TIME"]])
            if result_1:
                X[1]=1
            result_2=any([lst in list_en for lst in ["LONG"]])
            if result_2:
                X[2]=1 
    else:
        X=list(0 for i in range(0,N))
    return X 

def noun_chunk(x):
    a=[]
    for chunk in nlp(x).noun_chunks:
        a.append(chunk)
    return a

df['chunks'] = df['sents'].apply(lambda x: noun_chunk(x))

In [22]:
#df["ASO"]=df["Sentence"].apply(lambda y: satz_analytic2(nlp(y)))

In [23]:
df['Entities'] = df['Sentence'].apply(lambda x: entity(x))

In [24]:
#df['CNER'] = df['Entities'].apply(lambda x: NerFeature(x))

In [25]:
df.head(20)

,Chapter,ParaIndex,SentIndex,Paragraph,Sentence,Entities
0,1,0,0,Chapter 1,Chapter 1,[]
1,1,1,1,"On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.","On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.",[]
2,1,2,2,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...",The testimony of the ages confirms that the motions of the planets are orbicular.,[]
3,1,2,3,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...","Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles.",[]
4,1,2,4,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...","For among figures it is circles, and among bodies the heavens, that are considered the most perfect.",[]
5,1,2,5,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...","However, when experience seems to teach something different to those who pay careful attention, namely, that the planets deviate from a simple circular path, it gives rise to a powerful sense of wonder, which at length drives people to look into causes.","[(rise, ASTR)]"
6,1,3,6,"It is just this from which astronomy arose among humans. Astronomy's aim is considered to be to show why the stars' motions appear to be irregular on earth, despite their being exceedingly well ordered in heaven, and to investigate the specific circles whe...",It is just this from which astronomy arose among humans.,[]
7,1,3,7,"It is just this from which astronomy arose among humans. Astronomy's aim is considered to be to show why the stars' motions appear to be irregular on earth, despite their being exceedingly well ordered in heaven, and to investigate the specific circles whe...","Astronomy's aim is considered to be to show why the stars' motions appear to be irregular on earth, despite their being exceedingly well ordered in heaven, and to investigate the specific circles whereby the stars may be moved, so that by their aid the pos...","[(so, CAUS)]"
8,1,4,8,"Before the distinction between the first motion(1) and the second motions(2) was established, people noted (in contemplating the sun, moon and stars) that their diurnal paths were visually very nearly equivalent to circles. These were, however, entwined on...","Before the distinction between the first motion(1) and the second motions(2) was established, people noted (in contemplating the sun, moon and stars) that their diurnal paths were visually very nearly equivalent to circles.","[(diurnal, ASTR)]"
9,1,4,9,"Before the distinction between the first motion(1) and the second motions(2) was established, people noted (in contemplating the sun, moon and stars) that their diurnal paths were visually very nearly equivalent to circles. These were, however, entwined on...","These were, however, entwined one upon another like yarn on a ball, and the circles were for the most part smaller(3) circles of the sphere, rarely the greatest(4) (such","[(sphere, GEOM)]"


In [26]:
df.tail(10)


,Chapter,ParaIndex,SentIndex,Paragraph,Sentence,Entities
6689,appendix b,1602,6689,"Mars from Earth to be a little greater than half of the distance of the Sun from Earth. Kepler computed the diurnal latitudinal parallax of Mars to be 3' 28"", from which he determined the latitude of Mars without parallax to be 2° 25'. Since Mars's distanc...","Kepler determined the diurnal motions of the Sun and Mars to be 58' 38"" and 22' 36"" respectively, and the sum of the diurnal motions was 1° 21' 14"".","[(diurnal, ASTR), (Sun, PLAN), (Mars, PLAN), (58' 38"", COOR), (22' 36"", COOR), (diurnal, ASTR), (1° 21' 14"", COOR)]"
6690,appendix b,1602,6690,"Mars from Earth to be a little greater than half of the distance of the Sun from Earth. Kepler computed the diurnal latitudinal parallax of Mars to be 3' 28"", from which he determined the latitude of Mars without parallax to be 2° 25'. Since Mars's distanc...","With the sum of the diurnal motions of Mars and the Sun (1° 21' 14""), Kepler computed the time it takes for Mars and the Sun to move 58' 38"", that is, about 17 hours 20 minutes.","[(diurnal, ASTR), (Mars, PLAN), (Sun, PLAN), (1° 21' 14"", COOR), (Mars, PLAN), (Sun, PLAN), (58' 38"", COOR), (17 hours 20 minutes, TIME)]"
6691,appendix b,1603,6691,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...","As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system.","[(Sun, PLAN)]"
6692,appendix b,1603,6692,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...","The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively.","[(longitudes, ASTR), (Mars, PLAN), (Earth, PLAN), (Sun, PLAN), (198° 21' 30"", COOR), (199° 20' 8"", COOR)]"
6693,appendix b,1603,6693,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...","The deviation of elongation between Mars and the Earth remain 58' 38"".","[(elongation, ASTR), (Mars, PLAN), (Earth, PLAN), (58' 38"", COOR)]"
6694,appendix b,1603,6694,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...","To find the position of Mars at opposition, Kepler computed the angular distance that Mars and Earth—now substituting the place of the Sun—moved during 17 hours 20 minutes; Mars moved eastward about 16' 20"" and the Sun westward about 42' 18"".","[(Mars, PLAN), (opposition, ASTR), (Mars, PLAN), (Earth, PLAN), (Sun, PLAN), (17 hours 20 minutes, TIME), (Mars, PLAN), (eastward, ASTR), (16' 20"", COOR), (Sun, PLAN), (42' 18"", COOR)]"
6695,appendix b,1603,6695,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...","Accordingly, Kepler determined the longitude of Mars at opposition to be 198° 37' 50"" from which he subtracted about 39"" in order to correct Mars's orbit; he got 198° 37' 10"" (18° 37' 10"" Libra).","[(longitude, ASTR), (Mars, PLAN), (opposition, ASTR), (198° 37' 50"", COOR), (Mars, PLAN), (orbit, ASTR), (198° 37' 10"", COOR), (18° 37' 10"" Libra, COOR)]"
6696,appendix b,1604,6696,"The Sun moved 

def annot_observation_01(x):
    list_en=[]
    for idx in range(0,len(x)):
        list_en.append(x[idx][1])
    if any (lst in ("LONG","DATE","TIME") for lst in list_en):
        y=1
    else:
        y=0 
    return y

def annot_observation_02(x):
    list_en=[]
    for idx in range(0,len(x)):
        list_en.append(x[idx][1])
    result= all(lst in list_en for lst in ["LONG","DATE","TIME"])
    if result:
        #y=1
        y="Obs"
    else:
        y="NObs"
    return y
              

def annot_observation_03(x):
    list_en=[]
    for idx in range(0,len(x)):
        list_en.append(x[idx][1])
    result= all(lst in list_en for lst in ["DATE","TIME"])
    if result:
        y=1
    else:
        y=0
    return y

In [27]:
#df['Label'] = df['Entities'].apply(lambda x: annot_observation_02(x))

In [28]:
df.head()

,Chapter,ParaIndex,SentIndex,Paragraph,Sentence,Entities
0,1,0,0,Chapter 1,Chapter 1,[]
1,1,1,1,"On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.","On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.",[]
2,1,2,2,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...",The testimony of the ages confirms that the motions of the planets are orbicular.,[]
3,1,2,3,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...","Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles.",[]
4,1,2,4,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...","For among figures it is circles, and among bodies the heavens, that are considered the most perfect.",[]


In [29]:
df.tail(10)

,Chapter,ParaIndex,SentIndex,Paragraph,Sentence,Entities
6689,appendix b,1602,6689,"Mars from Earth to be a little greater than half of the distance of the Sun from Earth. Kepler computed the diurnal latitudinal parallax of Mars to be 3' 28"", from which he determined the latitude of Mars without parallax to be 2° 25'. Since Mars's distanc...","Kepler determined the diurnal motions of the Sun and Mars to be 58' 38"" and 22' 36"" respectively, and the sum of the diurnal motions was 1° 21' 14"".","[(diurnal, ASTR), (Sun, PLAN), (Mars, PLAN), (58' 38"", COOR), (22' 36"", COOR), (diurnal, ASTR), (1° 21' 14"", COOR)]"
6690,appendix b,1602,6690,"Mars from Earth to be a little greater than half of the distance of the Sun from Earth. Kepler computed the diurnal latitudinal parallax of Mars to be 3' 28"", from which he determined the latitude of Mars without parallax to be 2° 25'. Since Mars's distanc...","With the sum of the diurnal motions of Mars and the Sun (1° 21' 14""), Kepler computed the time it takes for Mars and the Sun to move 58' 38"", that is, about 17 hours 20 minutes.","[(diurnal, ASTR), (Mars, PLAN), (Sun, PLAN), (1° 21' 14"", COOR), (Mars, PLAN), (Sun, PLAN), (58' 38"", COOR), (17 hours 20 minutes, TIME)]"
6691,appendix b,1603,6691,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...","As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system.","[(Sun, PLAN)]"
6692,appendix b,1603,6692,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...","The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively.","[(longitudes, ASTR), (Mars, PLAN), (Earth, PLAN), (Sun, PLAN), (198° 21' 30"", COOR), (199° 20' 8"", COOR)]"
6693,appendix b,1603,6693,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...","The deviation of elongation between Mars and the Earth remain 58' 38"".","[(elongation, ASTR), (Mars, PLAN), (Earth, PLAN), (58' 38"", COOR)]"
6694,appendix b,1603,6694,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...","To find the position of Mars at opposition, Kepler computed the angular distance that Mars and Earth—now substituting the place of the Sun—moved during 17 hours 20 minutes; Mars moved eastward about 16' 20"" and the Sun westward about 42' 18"".","[(Mars, PLAN), (opposition, ASTR), (Mars, PLAN), (Earth, PLAN), (Sun, PLAN), (17 hours 20 minutes, TIME), (Mars, PLAN), (eastward, ASTR), (16' 20"", COOR), (Sun, PLAN), (42' 18"", COOR)]"
6695,appendix b,1603,6695,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system. The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively. The deviation of elongation between Mars and the...","Accordingly, Kepler determined the longitude of Mars at opposition to be 198° 37' 50"" from which he subtracted about 39"" in order to correct Mars's orbit; he got 198° 37' 10"" (18° 37' 10"" Libra).","[(longitude, ASTR), (Mars, PLAN), (opposition, ASTR), (198° 37' 50"", COOR), (Mars, PLAN), (orbit, ASTR), (198° 37' 10"", COOR), (18° 37' 10"" Libra, COOR)]"
6696,appendix b,1604,6696,"The Sun moved 

In [30]:
def show_chapter(x):
     return df.loc[df.Chapter==10]

In [31]:
len(df)

6699

In [32]:
interact(show_chapter,x=(1,70,1))      

interactive(children=(IntSlider(value=35, description='x', max=70, min=1), Output()), _dom_classes=('widget-in…

<function __main__.show_chapter(x)>

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6699 entries, 0 to 6698
Data columns (total 6 columns):
Chapter      6699 non-null object
ParaIndex    6699 non-null int64
SentIndex    6699 non-null int64
Paragraph    6699 non-null object
Sentence     6699 non-null object
Entities     6699 non-null object
dtypes: int64(2), object(4)
memory usage: 314.1+ KB


In [34]:
df.dtypes

Chapter      object
ParaIndex     int64
SentIndex     int64
Paragraph    object
Sentence     object
Entities     object
dtype: object

## Save the result as a JSON file

We have provided 3 different versions of annotation of observational sentences based on rule-based methods specifically using named entity recognition. These can be modified by a human annotator for having more precise labels. 
- A01--> If there is an entity from labels LONG, DATA, TIME in the sentences. we consider the sentence as an observational sentences 


- A02-->If there are all entities from labels LONG, DATA, TIME in the sentence. We consider the sentence as an observational sentence

-  A03 -->If there are all entities from labels  DATA, TIME in the sentence. We consider the sentence as an observational sentence

save the data without Observatioanl Column

In [35]:
df.to_json("dfV10A02.json")


In [36]:
df.to_pickle("dfV10A02.pkl")

In [37]:
#df.to_json("dfObsV01A01.json")
#df.to_json("dfObsR01V05A02.json")
#df.to_json("dfObsV01A03.json")

In [38]:
#df.to_pickle("dfObsR01V05A02.pickle")